## X-track plotting

### Imports

In [2]:
# Import standard libraries
import numpy as np
from cpymad.madx import Madx
import json
import xtrack as xt
import xfields as xf
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout
from ipywidgets import interact
from plotly.subplots import make_subplots

# Import plotting functions
from plotting_functions import return_plot_lattice_with_tracking

# Nicer plots
pio.templates.default = "plotly_white"

# Different backend
import plotly.io as pio
# pio.renderers.default = "vscode"

# Color list [blue, red, green, purple, orange,light blue, pink, light green, light violet, yellow]
l_colors = px.colors.qualitative.Plotly




### Load json files, build trackers and corresponding dataframes

In [3]:
# Load the lines into json files
with open("json_lines/line_b1.json", "r") as fid:
    dct_b1 = json.load(fid)
    line_b1 = xt.Line.from_dict(dct_b1)

with open("json_lines/line_b4.json", "r") as fid:
    dct_b4 = json.load(fid)
    line_b4 = xt.Line.from_dict(dct_b4)

# Build trackers
tracker_b1 = line_b1.build_tracker()
tracker_b4 = line_b4.build_tracker()

# Build a dataframe with the elements of the lines
df_elements_b1 = pd.DataFrame([x.to_dict() for x in line_b1.elements])
df_elements_b4 = pd.DataFrame([x.to_dict() for x in line_b4.elements])

df_elements_b1


Done loading line from dict.           
Done loading line from dict.           
generating ./c1cd91521abf4d1d994d852d45dc5f2a.c
the current directory is '/home/cdroin/xsuite-simulations'
generating ./3c335ed42e8846ae9707d0d53e28b607.c
the current directory is '/home/cdroin/xsuite-simulations'


,__class__,_dummy,length,order,inv_factorial_order,hxl,hyl,radiation_flag,knl,ksl,...,pn,ps,max_x,max_y,a_squ,b_squ,a_b_squ,cos_z,sin_z,angle
0,Marker,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marker,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Drift,NaN,21.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Multipole,NaN,1.7000,0.0,1.0,0.0,0.0,0.0,[0.0],[0.0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Drift,NaN,1.4905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23652,Drift,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23653,Drift,NaN,3.7445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23654,Multipole,NaN,1.7000,0.0,1.0,0.0,0.0,0.0,[-0.0],[0.0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23655,Drift,NaN,18.7000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get surveys and beam twiss parameters


In [4]:
# Get survey dataframes
df_sv_b1 = tracker_b1.survey().to_pandas()
df_sv_b4 = tracker_b4.survey().to_pandas()

# Get Twiss dataframes
tw_b1 = tracker_b1.twiss()
df_tw_b1 = tw_b1.to_pandas()
tw_b4 = tracker_b4.twiss()
df_tw_b4 = tw_b4.to_pandas()

# Reverse x-axis only for beam 1
df_sv_b1["X"] = -df_sv_b1["X"]
df_tw_b1["x"] = -df_tw_b1["x"]


### Add missing data to the dataframes of elements due to thin lens approximation

In [5]:
def return_dataframe_corrected_for_thin_lens_approx(df_elements, df_tw):
    df_elements_corrected = df_elements.copy(deep=True)

    # Add all thin lenses (length + strength)
    for i, row in df_tw.iterrows():
        # Correct for thin lens approximation and weird duplicates
        if ".." in row["name"] and 'f' not in row["name"].split("..")[1]:
            name = row["name"].split("..")[0]
            index = df_tw[df_tw.name == name].index[0]

            # Add length
            if np.isnan(df_elements_corrected.loc[index]["length"]):
                df_elements_corrected.at[index, "length"] = 0.0
            df_elements_corrected.at[index, "length"] += df_elements.loc[i]["length"]

            # Add strength
            if np.isnan(df_elements_corrected.loc[index]["knl"]).all():
                df_elements_corrected.at[index, "knl"] = (
                    np.array([0.0] * df_elements.loc[i]["knl"].shape[0], dtype=np.float64)
                    if type(df_elements.loc[i]["knl"]) != float
                    else 0.0
                )
            df_elements_corrected.at[index, "knl"] = (
                df_elements_corrected.loc[index, "knl"] + np.array(df_elements.loc[i]["knl"])
                if type(df_elements.loc[i]["knl"]) != float
                else df_elements.loc[i]["knl"]
            )

            # Replace order
            df_elements_corrected.at[index, "order"] = df_elements.loc[i]["order"]

            # Drop row
            df_elements_corrected.drop(i, inplace=True)

    return df_elements_corrected


df_elements_b1_corrected = return_dataframe_corrected_for_thin_lens_approx(df_elements_b1, df_tw_b1)
df_elements_b4_corrected = return_dataframe_corrected_for_thin_lens_approx(df_elements_b4, df_tw_b4)
df_elements_b1_corrected


,__class__,_dummy,length,order,inv_factorial_order,hxl,hyl,radiation_flag,knl,ksl,...,pn,ps,max_x,max_y,a_squ,b_squ,a_b_squ,cos_z,sin_z,angle
0,Marker,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Marker,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Drift,NaN,21.0300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Multipole,NaN,1.7000,0.0,1.0,0.0,0.0,0.0,[0.0],[0.0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Drift,NaN,1.4905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23652,Drift,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23653,Drift,NaN,3.7445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23654,Multipole,NaN,1.7000,0.0,1.0,0.0,0.0,0.0,[-0.0],[0.0],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23655,Drift,NaN,18.7000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get indices between IP8 and IP2, and IP4 and IP6


In [6]:
idx_ip4_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip4'].index[0]
idx_ip6_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip6'].index[0]
idx_ip8_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip8'].index[0]
idx_ip2_b1 = df_tw_b1.loc[df_tw_b1['name'] == 'ip2'].index[0]
l_indices_to_keep = list(range(idx_ip4_b1,idx_ip6_b1)) + list(range(idx_ip8_b1,idx_ip2_b1))

### Make interactive plot for the lattice

In [7]:
fig = return_plot_lattice_with_tracking(
    df_sv_b1,
    df_elements_b1_corrected,
    df_tw_b1,
    df_sv_4 = df_sv_b4,
    df_tw_4 = df_tw_b4,
    l_indices_to_keep=l_indices_to_keep)
    
config = {'displayModeBar': True, 'displaylogo': False, 'scrollZoom': True}
#fig.show(config=config)
#fig.write_html("layout.html")



### Make interactive plot to change knobs

In [8]:
# Initialize crossing angle at 0
tracker_b1.vars['on_x1'] = -0
tw_b1 = tracker_b1.twiss()
twmb19r5 = tw_b1.get_twiss_init(at_element='mb.b19l5.b1')
tw_part =  tracker_b1.twiss(ele_start='mb.b19l5.b1', ele_stop='mb.b19r5.b1', twiss_init=twmb19r5)

In [9]:
# Dropdown for the knob
dropdown = widgets.Dropdown(
    options=[(key, key) for key in tracker_b1.vars._owner.keys()
],
    value='on_x5',
    description='Knob',
)

# Slider for the knob
slider = widgets.BoundedFloatText(
    value=0.,
    min=-5000.,
    max=5000.,
    step=0.1,
    description='Knob value',
    continuous_update=False,
    style= {'description_width': 'initial', 'margin-left': '10px'},
    #layout=Layout(width='40%'),

)

# Build figure
fig = make_subplots(rows=3, cols=1)
fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['betx'],
        mode="lines",
        showlegend=True,
        name = r'$\beta_x$',
        legendgroup = '1',
    ), row=1, col=1)

fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['bety'],
        mode="lines",
        showlegend=True,
        name = r'$\beta_y$',
        legendgroup = '1',
    ), row=1, col=1)

fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['x'],
        mode="lines",
        showlegend=True,
        name = r'$x$',
        xaxis="x",
        yaxis="y2",
        legendgroup = '2',
    ), row=2, col=1)

fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['y'],
        mode="lines",
        showlegend=True,
        name = r'$y$',
        xaxis="x",
        yaxis="y2",
        legendgroup = '2',
    ), row=2, col=1)

fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['dx'],
        mode="lines",
        showlegend=True,
        name = r'$D_x$',
        xaxis="x",
        yaxis="y3",
        legendgroup = '3',
    ), row=3, col=1)

fig.append_trace(go.Scatter(
        x=tw_part['s'],
        y=tw_part['dy'],
        mode="lines",
        showlegend=True,
        name = r'$D_y$',
        xaxis="x",
        yaxis="y3",
        legendgroup = '3',
    ), row=3, col=1)

# Update overall layout
fig.update_layout(
    title_text="Transverse dynamics evolution with crossing angle",
    title_x=0.5,
    showlegend=True,
    xaxis_showgrid=True,
    yaxis_showgrid=True,
    #xaxis_title=r'$s$',
    #yaxis_title=r'$[m]$',
    width=1000,
    height=600,
    legend_tracegroupgap = 130,
)


# Update yaxis properties
fig.update_yaxes(title_text=r'$\beta_{x,y}$ [m]', range = [0, 25000],row=1, col=1)
fig.update_yaxes(title_text=r'(Closed orbit)$_{x,y}$ [m]', range = [-0.12, 0.12],row=2, col=1)
fig.update_yaxes(title_text=r'$D_{x,y}$ [m]', range = [-1.5, 2.5], row=3, col=1)
fig.update_xaxes(title_text=r'$s$',row=3, col=1)


g = go.FigureWidget(fig)



# Build interaction
def response(change):

    tracker_b1.vars[dropdown.value] = slider.value
    tw_b1 = tracker_b1.twiss()
    twmb19r5 = tw_b1.get_twiss_init(at_element='mb.b19l5.b1')
    tw_part =  tracker_b1.twiss(ele_start='mb.b19l5.b1', ele_stop='mb.b19r5.b1', twiss_init=twmb19r5)

    with g.batch_update():
        g.data[0].y = tw_part['betx']
        g.data[1].y = tw_part['bety']
        g.data[2].y = tw_part['x']
        g.data[3].y = tw_part['y']
        g.data[4].y = tw_part['dx']
        g.data[5].y = tw_part['dy']

container = widgets.HBox(children=[dropdown, slider,])

slider.observe(response, names="value")

config = {'displayModeBar': True, 'displaylogo': False}
#widgets.VBox([container, g])
#g.show(config=config)




In [10]:
i = 0
for item in vars(tracker_b1).items():
    print(item)
    i+=1
    if i > 10:
        break


('config', {'XTRACK_MULTIPOLE_NO_SYNRAD': True, 'XFIELDS_BB3D_NO_BEAMSTR': True})
('iscollective', False)
('_element_ref_data', None)
('_enable_pipeline_hold', False)
('global_xy_limit', 1.0)
('extra_headers', ())
('io_buffer', <BufferNumpy 1048568/1048576>)
('line', <xtrack.line.Line object at 0x7f6376758730>)
('_tracker_data', <xtrack.tracker_data.TrackerData object at 0x7f637c816e90>)
('num_elements', 23657)
('_buffer', <BufferNumpy 50080/2621440>)


In [71]:
#tracker_b1.element_refs['mbw.a6r3.b1'].knl[0]._info()


set_var = tracker_b1.element_refs['mbw.a6r3.b1'].knl[0]._expr._get_dependencies()
#tracker_b1.vars['ad34.lr3']._info(limit=48)
for var in set_var:
    name_var = str(var).split("'")[1]
    val = tracker_b1.vars[name_var ]._get_value()
    expr = tracker_b1.vars[name_var ]._expr
    dependencies = tracker_b1.vars[name_var ]._expr._get_dependencies()
    targets = tracker_b1.vars[name_var ]._find_dependant_targets()

print('name_var', name_var, 'val', val, 'expr', expr, 'dependencies', dependencies, 'targets', targets)

name_var ad34.lr3 val 0.00018872907808553264 expr ((vars['aip3']/3.0)*(1.0-vars['r0'])) dependencies {vars['ad34.lr3']} targets [vars['ad34.lr3'], element_refs['mbw.f6l3.b1'].knl, element_refs['mbw.b6l3.b1'].hxl, element_refs['mbw.a6l3.b1'].knl, element_refs['mbw.a6r3.b1'].knl, element_refs['mbw.d6r3.b1'], element_refs['mbw.e6r3.b1'].hxl, element_refs['mbw.c6l3.b1'].knl, element_refs['mbw.c6l3.b1'], element_refs['mbw.b6r3.b1'], element_refs['mbw.b6r3.b1'].hxl, element_refs['mbw.b6l3.b1'].knl[0], element_refs['mbw.d6r3.b1'].knl[0], element_refs['mbw.d6l3.b1'], element_refs['mbw.f6l3.b1'].knl[0], element_refs['mbw.a6r3.b1'].knl[0], element_refs['mbw.c6r3.b1'].knl, element_refs['mbw.d6l3.b1'].hxl, element_refs['mbw.e6l3.b1'].knl[0], element_refs['mbw.e6l3.b1'].hxl, element_refs['mbw.c6l3.b1'].knl[0], element_refs['mbw.a6l3.b1'].knl[0], element_refs['mbw.d6r3.b1'].hxl, element_refs['mbw.a6r3.b1'], element_refs['mbw.b6l3.b1'].knl, element_refs['mbw.b6l3.b1'], element_refs['mbw.e6r3.b1'].knl

In [73]:
name_var = 'ab.a45'
print(tracker_b1.vars[name_var]._expr._get_dependencies())

AttributeError: 'NoneType' object has no attribute '_get_dependencies'

In [45]:
print(['mb.b17r4.b1' in tracker_b1.element_refs._owner.keys()])


[True]


In [65]:
tracker_b1.element_refs['mb.b28r4.b1..1'].knl[0]._expr

(vars['ab.a45']/2.0)

In [ ]:
tracker_b1.vars['ad34.lr3']._info(limit=48)


In [ ]:
 x = tracker_b1.vars['on_x8']._find_dependant_targets()
 print(x)

In [ ]:
i = 0
for key in tracker_b1.vars._value.keys():#tracker_b1.element_refs._value.keys():
    print(key)
    if i>10:
        break

In [ ]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto

app = JupyterDash(__name__)

nodes = [
    {
        "data": {"id": short, "label": label},
        #"position": {"x": 20 * lat, "y": -20 * long},
    }
    for short, label, long, lat in (
        ("la", "Los Angeles", 34.03, -118.25),
        ("nyc", "New York", 40.71, -74),
        ("to", "Toronto", 43.65, -79.38),
        ("mtl", "Montreal", 45.50, -73.57),
        ("van", "Vancouver", 49.28, -123.12),
        ("chi", "Chicago", 41.88, -87.63),
        ("bos", "Boston", 42.36, -71.06),
        ("hou", "Houston", 29.76, -95.37),
    )
]

edges = [
    {"data": {"source": source, "target": target}}
    for source, target in (
        ("van", "la"),
        ("la", "chi"),
        ("hou", "chi"),
        ("to", "mtl"),
        ("mtl", "bos"),
        ("nyc", "bos"),
        ("to", "hou"),
        ("to", "nyc"),
        ("la", "nyc"),
        ("nyc", "bos"),
    )
]

elements = nodes + edges


app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dropdown-update-layout",
            value="grid",
            clearable=False,
            options=[
                {"label": name.capitalize(), "value": name}
                for name in ["grid", "random", "circle", "cose", "concentric"]
            ],
        ),
        cyto.Cytoscape(
            id="cytoscape-update-layout",
            layout={"name": "grid"},
            style={"width": "100%", "height": "450px"},
            elements=elements,
        ),
    ]
)


@app.callback(
    Output("cytoscape-update-layout", "layout"),
    Input("dropdown-update-layout", "value"),
)
def update_layout(layout):
    return {"name": layout, "animate": True}


# if __name__ == "__main__":
#     app.run_server(debug=True, port = 8054)

In [ ]:
from dash import Dash, html, dcc, Input, Output
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto

app = JupyterDash(__name__)

In [ ]:
nodes = []
edges = []
set_nodes = set()
set_edges = set()
# Browse all the vars
for key in tracker_b1.vars._value.keys():
    if key not in set_nodes:
        # Add the node
        nodes.append(
            {
                "data": {"id": key, "label": key},
            }
        )
        # Add node to the set 
        set_nodes.add(key)
    
    # Add all children
    for child in tracker_b1.vars[key]._find_dependant_targets():
        if child not in set_nodes:
            # Add the node
            nodes.append(
                {
                    "data": {"id": child, "label": child},
                }
            )
            # Add node to the set 
            set_nodes.add(child)
        if (key, child) not in set_edges:
            # Add the edge
            edges.append(
                {"data": {"source": key, "target": child}}
            )
            # Add edge to the set
            set_edges.add((key, child))

        # # Add subsequent grandchildren
        # for grandchild in tracker_b1.vars[child]._find_dependant_targets():
        #     if grandchild not in set_nodes:
        #         # Add the node
        #         nodes.append(
        #             {
        #                 "data": {"id": grandchild, "label": grandchild},
        #             }
        #         )
        #         # Add node to the set 
        #         set_nodes.add(grandchild)
        #     edges.append(
        #         {"data": {"source": child, "target": grandchild}}
        #     )



In [ ]:
elements = nodes + edges


app.layout = html.Div(
    [
        dcc.Dropdown(
            id="dropdown-update-layout",
            value="grid",
            clearable=False,
            options=[
                {"label": name.capitalize(), "value": name}
                for name in ["grid", "random", "circle", "cose", "concentric"]
            ],
        ),
        cyto.Cytoscape(
            id="cytoscape-update-layout",
            layout={"name": "grid"},
            style={"width": "100%", "height": "450px"},
            elements=elements,
        ),
    ]
)


@app.callback(
    Output("cytoscape-update-layout", "layout"),
    Input("dropdown-update-layout", "value"),
)
def update_layout(layout):
    return {"name": layout, "animate": True}


if __name__ == "__main__":
    app.run_server(debug=True, port = 8054)